# Homework  Task

In module 9 we learned how to solve a problem from end to end using SKLearn and pipelines. There we learned how to  predict house prices in California.
In this homework, we adopt this end to end pipeline to tackle the Titanic Survival problem.
Adopt the machine learning pipeline above to tackle the Titanic problem. 
## Submission instructions
1. Before completing this homework,
    * please review this homework's submission form on Canvas available under the "Modules" menu option and 
    * briefly review this notebook end to end. 
* To get you started we provide a template solution with missing code and prompts. Please complete the missing code, run the experiments and log your results.
* When you're sufficiently happy with your results, please begin the submission process on Canvas. Use the submission form for this homework available under "Modules" menu option. Please note that the submission form is available at the same place where you downloaded the homework from.
* You may wish to reference this quiz while working through the tasks.

You will need to:

* __Important:__ To ensure the accuracy of your results (for the quiz) you must split the data using a 20% test set with random seed = 42.
* __EDA.__ Identify the types of data available, evaluate basic statistical information about the data and determine whether you have any missing or misformated data.

* __Feature Engineering.__  Develop at least one new feature.  The following [webpage](http://trevorstephens.com/kaggle-titanic-tutorial/r-part-4-feature-engineering/) has some great ideas for creating new features on the Titanic dataset.
* __Pre-processing.__  All work must be performed using pipelines.  You can adapt code from above or develop your own.
* __Modeling.__Evaluate at least two appropriate algorithms (estimators) for generating predictions.
    * Use grid search to tune hyperparameters.
    * Use crossfold evaluation (cv=5).
* __Evaluation.__ Select appropriate metrics for the problem to evaluate your models.
* __Reporting.__ Record all experiments in a table of results (pandas dataframe) including at least the following information:  
    * description of the model (algorithim, notable processing steps) 
    * key hyperparameters
    * results (using one or more appropriate metrics)
    * run time for each experiment (train and test results)
    * hardware used
* __Analysis__. Perform a significance test on your best models and discuss results (see Module 09.5 in Canvas for a video lecture on significance testing).

Your final pipeline will take the following format:

<PRE>
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),  # combination of numerical and categorical pipelines
        ("logRegression", LogisticRegression())  # replace with whatever estimator(s) you are using
    ])

</PRE>


# Prepare the notebook (imports, helper functions, etc.)

In [ ]:
# imports
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import re
from time import time
from scipy import stats
import json

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
# Imports for metrics
from sklearn.model_selection import cross_val_score, ShuffleSplit
# Imports for stats
from scipy import stats
                       
# Convert a number to a percent.    
def pct(x):
    return round(100*x,1)

# Set up reporting
import pandas as pd
results = pd.DataFrame(columns=["ExpID", "Cross fold train accuracy", "Test Accuracy", "p-value", "Train Time(s)", "Test Time(s)", "Experiment description"])

# Set up ShuffleSplit for p_value testing
cv = ShuffleSplit(n_splits=30, test_size=0.3, random_state=0)

In [ ]:
def ttest(control, treatment):
    #paired t-test; two-tailed p-value      A   ,    B
    (t_score, p_value) = stats.ttest_rel(control, treatment)

    if p_value > 0.05/2:  #Two sided 
        print('There is no significant difference between the two machine learning pipelines (Accept H0)')
    else:
        print('The two machine learning pipelines are different (reject H0) \n(t_score, p_value) = (%.2f, %.5f)'%(t_score, p_value) )
        if t_score > 0.0: #in the case of regression lower RMSE is better; A is lower 
            print('Machine learning pipeline A is better than B')
        else:
            print('Machine learning pipeline B is better than A')
    return p_value

# Data import and EDA

In [ ]:
data = pd.read_csv('datasets/titanic/train.csv')
y = data['Survived']
x = data.drop(['Survived', 'Ticket', 'Cabin'], axis = 1)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
# Enter your code here to find attribute with most null values. 
#==================================================#
#               Your code starts here              #
#==================================================#

#==================================================#
#               Your code ends here                #
#               Please don't add code below here   #
#==================================================#

In [ ]:
data.describe()

In [ ]:
# Show the correlation of the Survived column with the features
corr_matrix = data.corr()
corr_matrix["Survived"].sort_values(ascending=False)

## Correlation Observations
The goal of this section is to get familiar with the data that will be used for the end to end pipeline. It is very important to explore the data and summarize its main characteristics before diving in the machine learning models. It is also interesting to see how the different features are correlated with the target feature

* *Fare* and *Pclass* are the most highly correlated with survivorship. A higher fare indicates a higher chance of surviving; a lower class (where "first" class is better than "third") indicates a higher chance of survinging. 
* *Sex* and *Embarked* do not appear in the correlation list, because they are not numeric values. We can one-hot encode this column and re-examine later.

In [ ]:
from pandas.plotting import scatter_matrix

data = pd.read_csv('datasets/titanic/train.csv')

# Top four correlated inputs with survived
attributes = ["Survived", "Fare", "Pclass", "Parch", "Age"]
scatter_matrix(data[attributes], figsize=(12, 8));

## Evaluate categorical features with respect to Survived

In [ ]:
data = pd.read_csv('datasets/titanic/train.csv')
cat_vars = ['Sex', 'Pclass', 'Embarked']
plt.figure(figsize=(15,4))
for idx, cat in enumerate(cat_vars):
    plt.subplot(1, 3, idx+1)
    sns.countplot(data[cat], hue=data['Survived'])

In [ ]:
np.random.seed(42)
data = pd.read_csv('datasets/titanic/train.csv')
y = data['Survived']
x = data.drop(['Survived', 'Ticket', 'Cabin'], axis = 1)
# split 20% test data with random seed set to 42 for correct results
# and shuffle the dataset
#==================================================#
#               Your code starts here              #
#==================================================#
x_train, x_test, y_train, y_test = train_test_split()  
#==================================================#
#               Your code ends here                #
#               Please don't add code below here   #
#==================================================#

# Build processing pipelines
In this part of the homework the focus is on constructing the pipeline. Since the data has both numerical and categorical features, it is required to create two pipelines (one for each category of data) because they require different transformations. After finishing that, the two pipelines should be unified to produce one full pipeline that performs transformation on all the dataset

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion

# Identify the numeric features we wish to consider. 
num_attribs = [
    'Age', 
    'Parch', 
    'SibSp',
    'Fare'
]
# Create a pipeline for the numeric features.
# Use DataFrameSelector with the numeric features defined above
# Use StandardScaler() to standardize the data
# Missing values will be imputed using the feature median.
#==================================================#
#               Your code starts here              #
#==================================================#
num_pipeline =Pipeline()   
#==================================================#
#               Your code ends here                #
#               Please don't add code below here   #
#==================================================#


# Identify the categorical features we wish to consider.
cat_attribs = [
    "Embarked", 
    "Sex",
    "Pclass"
]
# Identiy the range of expected values for the categorical features.
cat_values = [
    ['S','C','Q'],     # Embarked
    ['female','male'], # Sex
    [1,2,3] # Pclass
]
# Create a pipelne for the categorical features.
# Entries with missing values or values that don't exist in the range
# defined above will be one hot encoded as zeroes.
cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(sparse=False, handle_unknown="ignore"))
    ])

# Union the transformed, scaled numeric and categorical features.
#==================================================#
#               Your code starts here              #
#==================================================#
full_pipeline = FeatureUnion()     #TODO <- ColumnTransformer
#==================================================#
#               Your code ends here                #
#               Please don't add code below here   #
#==================================================#



# Feature Engineering
When conducting an end to end Machine Learning project, after exploring and preprocessing the data it is essential to think of feature engineering. It consists of creating new feature(s) based on the features that already exist in the dataset that can be useful for training the model.

## Develop a new feature based on the title from each passengers name

### Evaluate the distribution of titles

In [ ]:
data = pd.read_csv('datasets/titanic/train.csv')

data['Title'] = data['Name'].apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1))
sns.countplot(x='Title', data=data)
plt.title('Distribution of Titles Amongst Passengers')
plt.xticks(rotation=90);

### Consolidate special and miscellaneous titles
Most titles fall into one of four categories (Mr, Mrs., Miss, Master).  All other titles can be combined with one of these groups (i.e., they are French varients) or grouped as "other". 

In [ ]:
data['Title'] = data['Title'].replace({'Mlle':'Miss', 'Mme':'Mrs', 'Ms':'Miss'})
data['Title'] = data['Title'].replace(['Don', 'Dona', 'Rev', 'Dr','Major', 'Lady', 'Sir', 
                                       'Col', 'Capt', 'Countess', 'Jonkheer'],'Other')
sns.countplot(x='Title', data=data);
plt.title('Distribution of Titles After Consolidation');
data['Title'].value_counts()

## Create a transformer to perfom this new feature transformation in a pipeline

In [ ]:
class TitleAdder(BaseEstimator, TransformerMixin):
    def __init__(self, features=None): # no *args or **kargs
        self.features = features
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        df = pd.DataFrame(X, columns=self.features)
        df['Title'] = df['Name'].apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1))
        # Apply the necessary transformations to obtain the 5 title categories
        # (Mr, Mrs, Miss, Master, Other) like it was done in section 5.1.2
        #==================================================#
        #               Your code starts here              #
        #==================================================#
        df['Title'] = df['Title'].replace()
        df['Title'] = df['Title'].replace()                
        #==================================================#
        #               Your code ends here                #
        #               Please don't add code below here   #
        #==================================================#
        df.drop('Name', axis=1, inplace=True)
        return np.array(df.values)

# Modeling
Now that we have explored the data, cleaned it, preprocessed it and added a new feature to it, we can start the modeling part of the project by applying Machine Learning algorithms. In this section, you will have a baseline logistic regression model and grid searches on different models. In the end, you will find out which parameters are the best for each algorithm and you will be able to compare the performance of the models with the baseline model.

## Baseline Logistic Regression

In [ ]:
# use full pipeline above to build full pipeline with predictor
np.random.seed(42)
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LogisticRegression(random_state=42))
    ])

data = pd.read_csv('datasets/titanic/train.csv')
y = data['Survived']
x = data.drop(['Survived', 'Ticket', 'Cabin'], axis = 1)

# split 20% test data with random seed set to 42 for correct results
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, shuffle=True, random_state=42)

In [ ]:
start = time()
full_pipeline_with_predictor.fit(x_train, y_train)
np.random.seed(42)

# Set up cross validation scores 
# Use ShuffleSplit() with 30 splits, 30% test_size 
# and a random seed of 0
#==================================================#
#               Your code starts here              #
#==================================================#
cv30Splits = ShuffleSplit()
logit_scores = cross_val_score()     
#==================================================#
#               Your code ends here                #
#               Please don't add code below here   #
#==================================================#
logit_score_train = logit_scores.mean()
train_time = np.round(time() - start, 4)

# Time and score test predictions
start = time()
logit_score_test  = full_pipeline_with_predictor.score(x_test, y_test)
test_time = np.round(time() - start, 4)

In [ ]:
results.loc[0] = ["Baseline", pct(logit_score_train), np.round(pct(logit_score_test),3), 
                  "---", train_time, test_time, "Untuned LogisticRegression"]
results

## Decision Trees

Now that we have explored the data, cleaned it, preprocessed it, and added a new feature to it, we can start the modeling part of the project by applying Machine Learning algorithms. In this section we apply decision tree and random forests.

Decision Tree is one of the popular and most widely used Machine Learning Algorithms because of its robustness to noise, tolerance against missing information, handling of irrelevant, redundant predictive attribute values, low computational cost, interpretability, fast run time and robust predictors. 

#### Cost functions used for classification and regression.
In both cases the cost functions try to find most homogeneous branches, or branches having groups with similar responses. 

Regression : sum(y — prediction)²

Classification : G = sum(pk * (1 — pk))

A Gini score gives an idea of how good a split is by how mixed the response classes are in the groups created by the split. Here, pk is proportion of same class inputs present in a particular group.

#### DecisionTreeClassifier

The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain.
Information gain uses the entropy measure as the impurity measure and splits a node such that it gives the most amount of information gain. Whereas Gini Impurity measures the divergences between the probability distributions of the target attribute’s values and splits a node such that it gives the least amount of impurity.

**Gini** :  $\Large 1 - \sum^m_{i=1}(P_j^2)$

**Entropy** : $\Large \sum^m_{i=1}\left(P_j\cdot\:\log\:\left(P_j\right)\:)\right)$

#### DecisionTreeRegressor

The decision of making strategic splits heavily affects a tree’s accuracy. The decision criteria is different for classification and regression trees.Decision trees regression normally use mean squared error (MSE) to decide to split a node in two or more sub-nodes. 

Suppose we are doing a binary tree the algorithm first will pick a value, and split the data into two subset. For each subset, it will calculate the MSE separately. The tree chooses the value with results in smallest MSE value.

${MSE}=\frac{1}{m} \sum_{i=1}^m  (x_{i}-y_{i})^2 $

#### Feature importance formula
To calculate the importance of each feature, we will mention the decision point itself and its child nodes as well. The following formula covers the calculation of feature importance. 

For each decision tree, Scikit-learn calculates a nodes importance using Gini Importance, assuming only two child nodes (binary tree):

$\Large ni_j = w_jC_j - w_{left(j)}C_{left(j)} - w_{right(j)}C_{right(j)}$  

**Where**

ni_j= the importance of node j

w_j = weighted number of samples reaching node j

C_j= the impurity value of node j

left(j) = child node from left split on node j

right(j) = child node from right split on node j

The importance for each feature on a decision tree is then calculated as:

$\Large fi_i = \frac{\sum_{j:node \hspace{0.1cm} j \hspace{0.1cm} splits \hspace{0.1cm} on \hspace{0.1cm} feature \hspace{0.1cm} i}ni_j}{\sum_{k \hspace{0.1cm} \epsilon \hspace{0.1cm} all \hspace{0.1cm} nodes }ni_k}$

These can then be normalized to a value between 0 and 1 by dividing by the sum of all feature importance values:

$\Large normfi_i = \frac{fi_i}{\sum_{j \hspace{0.1cm}  \epsilon \hspace{0.1cm} all \hspace{0.1cm} features}fi_j}$

#### Decision Trees Parameters for classification

**criterion{“gini”, “entropy”}, default=”gini”** :The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain.

**max_depth, default=None** : The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

**min_samples_leaf int or float, default=1** : The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.

#### Decision Trees Parameters for Regression

**criterion{“mse”, “friedman_mse”, “mae”, “poisson”}, default=”mse”** : The function to measure the quality of a split. 

**max_depth, default=None** : The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

**min_samples_leaf int or float, default=1** : The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.

#### Random Forest Parameters

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. 

**n_estimatorsint, default=100** : The number of trees in the forest.

**max_depthint, default=None** : The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

**max_features** : The number of features to consider when looking for the best split.

**min_impurity_decreasefloat, default=0.0** : Threshold for early stopping in tree growth. A node will be split if this split induces a decrease of the impurity greater than or equal to this value.

**bootstrapbool, default=True** : Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.

### Regression Code

#### Decision Tree

In [ ]:
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

boston = load_boston()
data=pd.DataFrame(boston.data)
data.columns = boston.feature_names
data['PRICE']=boston.target
X_boston,y_boston = data.iloc[:,:-1],data.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train_boston,X_test_boston,y_train_boston,y_test_boston = train_test_split(X_boston,y_boston,test_size=0.2,random_state=42)

# Initialize and fit regressor

tree = DecisionTreeRegressor(criterion='mse',max_depth=4,random_state = 42)
tree.fit(X_train_boston, y_train_boston)

y_preds_boston = tree.predict(X_test_boston)

rmse = np.sqrt(mean_squared_error(y_test_boston,y_preds_boston))
print ('RMSE=%f'% (rmse))

In [ ]:
features = X_train_boston.columns
importances = tree.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.grid()
plt.show();

#### Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error

boston = load_boston()
data=pd.DataFrame(boston.data)
data.columns = boston.feature_names
data['PRICE']=boston.target
X_boston,y_boston = data.iloc[:,:-1],data.iloc[:,-1]
X_train_boston,X_test_boston,y_train_boston,y_test_boston = train_test_split(X_boston,y_boston,test_size=0.2,random_state=42)

RF = RandomForestRegressor(random_state = 42,n_estimators=100,criterion='mse',max_depth=6)

RF.fit(X_train_boston, y_train_boston)

y_preds_boston = RF.predict(X_test_boston)

rmse = np.sqrt(mean_squared_error(y_test_boston,y_preds_boston))
print ('RMSE=%f'% (rmse))

In [ ]:
features = X_train_boston.columns
importances = RF.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.grid()

plt.show();

### Classification Code

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# A Function to execute the grid search and record the results.
def ConductGridSearch(X_train, y_train, X_test, y_test, i=0, prefix='', n_jobs=-1,verbose=1):
    # Create a list of classifiers for our grid search experiment
    classifiers = [
        ('DecisionTrees', DecisionTreeClassifier(random_state=42)),
        ('RandomForest', RandomForestClassifier(random_state=42))
        
    ]

    # Arrange grid search parameters for each classifier
    params_grid = {
        'DecisionTrees' : {
            'criterion':['gini','entropy'],
            'max_depth':range(1,10),
            
            'min_samples_leaf':range(1,5)
        },
        
       'RandomForest':  {
            'max_depth': [9, 15, 22,],
            'max_features': [1, 3, 5],
            'min_samples_leaf': [3, 5, 10],
            'min_impurity_decrease':[0,1e-3,1e-4,1e-6],
            'bootstrap': [True],
            'n_estimators':[ 100, 150, 200]}
    }
    
    for (name, classifier) in classifiers:
        i += 1
        # Print classifier and parameters
        print('****** START',prefix, name,'*****')
        parameters = params_grid[name]
        print("Parameters:")
        for p in sorted(parameters.keys()):
            print("\t"+str(p)+": "+ str(parameters[p]))
        
        # generate the pipeline
        full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("predictor", classifier)
        ])
        
        # Execute the grid search
        params = {}
        for p in parameters.keys():
            pipe_key = 'predictor__'+str(p)
            params[pipe_key] = parameters[p] 
        grid_search = GridSearchCV(full_pipeline_with_predictor, params, scoring='accuracy', cv=5, 
                                   n_jobs=n_jobs, verbose=verbose)
        grid_search.fit(X_train, y_train)
                
        # Best estimator score
        best_train = pct(grid_search.best_score_)

        # Best estimator fitting time
        start = time()
        grid_search.best_estimator_.fit(X_train, y_train)
        train_time = round(time() - start, 4)
        
#         plt.barh(['Age', 'Parch', 'SibSp','Fare','x0_C' ,'x0_Q' ,'x0_S' ,'x1_female', 'x1_male' ,'x2_1' ,'x2_2', 'x2_3'], grid_search.best_estimator_.named_steps["predictor"].feature_importances_)
#         plt.show()
        features = ['Age', 'Parch', 'SibSp','Fare','x0_C' ,'x0_Q' ,'x0_S' ,'x1_female', 'x1_male' ,'x2_1' ,'x2_2', 'x2_3']
        importances = grid_search.best_estimator_.named_steps["predictor"].feature_importances_
        indices = np.argsort(importances)

        plt.title('Feature Importances')
        plt.barh(range(len(indices)), importances[indices], color='b', align='center')
        plt.yticks(range(len(indices)), [features[i] for i in indices])
        plt.xlabel('Relative Importance')
        plt.grid()
        plt.show();
        # Best estimator prediction time
        start = time()
        best_test_accuracy = pct(grid_search.best_estimator_.score(X_test, y_test))
        test_time = round(time() - start, 4)

        # Generate 30 training accuracy scores with the best estimator and 30-split CV
        # To calculate the best_train_accuracy use the pct() and mean() methods
        #==================================================#
        #               Your code starts here              #
        #==================================================#
        best_train_scores = cross_val_score(..., cv=cv30Splits)
        best_train_accuracy = pct()
        #==================================================#
        #               Your code ends here                #
        #               Please don't add code below here   #
        #==================================================#    
       
        # Conduct t-test with baseline logit (control) and best estimator (experiment)
        (t_stat, p_value) = stats.ttest_rel(logit_scores, best_train_scores)
        
        # Collect the best parameters found by the grid search
        print("Best Parameters:")
        best_parameters = grid_search.best_estimator_.get_params()
        param_dump = []
        for param_name in sorted(params.keys()):
            param_dump.append((param_name, best_parameters[param_name]))
            print("\t"+str(param_name)+": " + str(best_parameters[param_name]))
        print("****** FINISH",prefix,name," *****")
        print("")
        
        # Record the results
        results.loc[i] = [prefix+name, best_train_accuracy, best_test_accuracy, round(p_value,5), train_time, test_time, json.dumps(param_dump)]

In [ ]:
%%time
# This might take a while
if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    # n_jobs=-1 means that the computation will be dispatched on all the CPUs of the computer.
    #
    # By default, the GridSearchCV uses a 3-fold cross-validation. However, if it 
    #            detects that a classifier is passed, rather than a regressor, it uses a stratified 3-fold.
    
    data = pd.read_csv('datasets/titanic/train.csv')
    y = data['Survived']
    x = data.drop(['Survived', 'Ticket', 'Cabin'], axis = 1)

    # split 20% test data with random seed set to 42 for correct results
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, shuffle=True, random_state=42)
    ConductGridSearch(x_train, y_train, x_test, y_test, 0, "Best Model:",  n_jobs=-1,verbose=1)

In [ ]:
results

## Conduct Grid Search using a variety of Classification Algorithms

In this section, we're going to try to answer the following questions
Note: These questions should be answered in Canvas once you have completed this section.
### Best train accuracy
    Please submit the code snippet you added to calculate the best train accuracy in the section below of the notebook
### Best parameters for Logistic Regression 
    Based on the results obtained after conducting Grid Search in this section , choose the best parameters for Logistic Regression
### Best parameters for k-nearest neighbors
    Based on the results obtained after conducting Grid Search in this section, what is  the best parameter for n_neighbors for k-nearest neighbors?
### SVM Test Accuracy
Please enter the calculated value for test Accuracy of Support vector model in the section below of the notebook. 
(Report your number to 1 decimal point of precision. For example: 2.5)
### Statistical significance
    Which one of the models listed below is the most statistically significant based on the results of this section? 
    * Naive Bayes
    * Stochastic GD
    * RandomForest
    * Logistic Regression
### Choosing the best model
    Given the results that you obtained for the different models in this section, based on what information would you choose the best model to deploy? 
    * Cross fold Train Accuracy
    * Test Accuracy
    * p-value
    * Train Time
    * Test Time
 
    

In [ ]:

# A Function to execute the grid search and record the results.
def ConductGridSearch(X_train, y_train, X_test, y_test, i=0, prefix='', n_jobs=-1,verbose=1):
    # Create a list of classifiers for our grid search experiment
    classifiers = [
        ('Logistic Regression', LogisticRegression(random_state=42)),
        ('K-Nearest Neighbors', KNeighborsClassifier()),
        ('Naive Bayes', GaussianNB()),
        ('Support Vector', SVC(random_state=42)),
        ('Stochastic GD', SGDClassifier(random_state=42)),
        ('RandomForest', RandomForestClassifier(random_state=42)),
    ]

    # Arrange grid search parameters for each classifier
    params_grid = {
        'Logistic Regression': {
            'penalty': ('l1', 'l2'),
            'tol': (0.0001, 0.00001, 0.0000001), 
            'C': (10, 1, 0.1, 0.01),
        },
        'K-Nearest Neighbors': {
            'n_neighbors': (3, 5, 7, 8, 11),
            'p': (1,2),
        },
        'Naive Bayes': {},
        'Support Vector' : {
            'kernel': ('rbf', 'poly'),     
            'degree': (1, 2, 3, 4, 5),
            'C': (10, 1, 0.1, 0.01),
        },
        'Stochastic GD': {
            'loss': ('hinge', 'perceptron', 'log'),
            'penalty': ('l1', 'l2', 'elasticnet'),
            'tol': (0.0001, 0.00001, 0.0000001), 
            'alpha': (0.1, 0.01, 0.001, 0.0001), 
        },
        'RandomForest':  {
            'max_depth': [9, 15, 22, 26, 30],
            'max_features': [1, 3, 5],
            'min_samples_split': [5, 10, 15],
            'min_samples_leaf': [3, 5, 10],
            'bootstrap': [False],
            'n_estimators':[20, 80, 150, 200, 300]},
    }
    
    for (name, classifier) in classifiers:
        i += 1
        # Print classifier and parameters
        print('****** START',prefix, name,'*****')
        parameters = params_grid[name]
        print("Parameters:")
        for p in sorted(parameters.keys()):
            print("\t"+str(p)+": "+ str(parameters[p]))
        
        # generate the pipeline
        full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("predictor", classifier)
        ])
        
        # Execute the grid search
        params = {}
        for p in parameters.keys():
            pipe_key = 'predictor__'+str(p)
            params[pipe_key] = parameters[p] 
        grid_search = GridSearchCV(full_pipeline_with_predictor, params, scoring='accuracy', cv=5, 
                                   n_jobs=n_jobs, verbose=verbose)
        grid_search.fit(X_train, y_train)
                
        # Best estimator score
        best_train = pct(grid_search.best_score_)

        # Best estimator fitting time
        start = time()
        grid_search.best_estimator_.fit(X_train, y_train)
        train_time = round(time() - start, 4)

        # Best estimator prediction time
        start = time()
        best_test_accuracy = pct(grid_search.best_estimator_.score(X_test, y_test))
        test_time = round(time() - start, 4)

        # Generate 30 training accuracy scores with the best estimator and 30-split CV
        # To calculate the best_train_accuracy use the pct() and mean() methods
        #==================================================#
        #               Your code starts here              #
        #==================================================#
        best_train_scores = cross_val_score( ...., cv=cv30Splits)
        best_train_accuracy = pct()  
        #=================================================#
        #               Your code ends here                #
        #               Please don't add code below here   #
        #==================================================#    
       
        # Conduct t-test with baseline logit (control) and best estimator (experiment)
        (t_stat, p_value) = stats.ttest_rel(logit_scores, best_train_scores)
        
        # Collect the best parameters found by the grid search
        print("Best Parameters:")
        best_parameters = grid_search.best_estimator_.get_params()
        param_dump = []
        for param_name in sorted(params.keys()):
            param_dump.append((param_name, best_parameters[param_name]))
            print("\t"+str(param_name)+": " + str(best_parameters[param_name]))
        print("****** FINISH",prefix,name," *****")
        print("")
        
        # Record the results
        results.loc[i] = [prefix+name, best_train_accuracy, best_test_accuracy, round(p_value,5), train_time, test_time, json.dumps(param_dump)]

In [ ]:
%%time
# This might take a while
if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    # n_jobs=-1 means that the computation will be dispatched on all the CPUs of the computer.
    #
    # By default, the GridSearchCV uses a 3-fold cross-validation. However, if it 
    #            detects that a classifier is passed, rather than a regressor, it uses a stratified 3-fold.
    data = pd.read_csv('datasets/titanic/train.csv')
    y = data['Survived']
    x = data.drop(['Survived', 'Ticket', 'Cabin'], axis = 1)

    # split 20% test data with random seed set to 42 for correct results
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, shuffle=True, random_state=42)
    ConductGridSearch(x_train, y_train, x_test, y_test, 0, "Best Model:",  n_jobs=-1,verbose=1)

In [ ]:
results

## Observations

* The Support Vector Classifier with parameters [C=10, degree=2, kernel=poly] had the best overall performance followed by the Random Forest classifier. The p-values of 0.00057 and 0.0001, respectively, indicate that both results are signficantly different from the baseline Logistic Regression model. 
* There tuned Logistic Regression was not statistically different than the baseline (untuned) model.

# OPTIONAL: TMDB Box Office Prediction on Kaggle

 For this part of the homework 
You will need to look at the  [TMDB Box Office Prediction Competition on Kaggle](https://www.kaggle.com/c/tmdb-box-office-prediction)
Through the lectures and lab we learned how to predict house prices in California. In this homework, we adopted  the end to end pipeline to tackle the Titanic Survival problem. Now, we want you to adopt the machine learning pipeline to tackle the TMDB Box Office Prediction

In this dataset, you are provided with 7,398  movies (3,000 for training and 4,398 for testing) and a variety of metadata obtained from The Movie Database (TMDB). Movies are labeled with id. Data points include cast, crew, plot keywords, budget, posters, release dates, languages, production companies, and countries.

Feel free to use the [Kaggle API](https://github.com/Kaggle/kaggle-api) for downloading the dataset or submitting to the competition. It is not mandatory to use the package but it would be interesting to explore.
   You will need to: 
* __Important:__ Make sure your results are reproducible 
* __Important:__ Use the training data set provided by the competition to create a training set(70%), validation set (15%) and a test set (15%)
* __EDA.__ Identify the types of data available, evaluate basic statistical information about the data and determine whether you have any missing or misformated data.

* __Feature Engineering.__  Develop at least one new feature based on the existing features of the dataset
* __Pre-processing.__  All work must be performed using pipelines.  You can adapt code from above or develop your own.
* __Modeling.__Evaluate at least two appropriate algorithms (estimators) for generating predictions.
    * Use grid search to tune hyperparameters.
    * Use crossfold evaluation (cv=5).
* __Evaluation.__ Select appropriate metrics for the problem to evaluate your models.
* __Reporting.__ Record all experiments in a table of results (pandas dataframe) including at least the following information:  
    * description of the model (algorithim, notable processing steps) 
    * key hyperparameters
    * results (using one or more appropriate metrics)
    * run time for each experiment (train and test results)
    * hardware used
* __Analysis__. Perform a significance test on your best models and discuss results (see Module 09.5 in Canvas for a video lecture on significance testing).
* __Submit your best model to Kaggle__ Provide a screenshot of the kaggle submission
* __Comment your code and provide explanations of how you're proceeding in each part__


## Data Import & notebook preperation


In [ ]:
import pandas as pd


In [ ]:
data = pd.read_csv('datasets/TMDB/train.csv')
y = data ['revenue']
x = data .drop(['revenue'], axis = 1)
test_data = pd.read_csv('datasets/TMDB/test.csv')


* __Important:__ Remember that x and y should be split into a training set (70% of the original dataset), a validation set (15% of the original dataset) and a test set (15% of the original dataset. 
* test_data will be only used for the kaggle submission

## EDA

## Feature engineering

## Preprocessing


## Modeling

## Evaluation, reporting and analysis


## Kaggle Submission
